In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
from utils import preprocessing, transforms
from utils.constants import PLOTCOLORS
import matplotlib.pyplot as plt
from scipy import stats

RANDOM_STATE = 21

cwd = Path.cwd()
husky_data = cwd / "borealtc-data"
vulpi_data = cwd / "data"
figs_path = cwd / "analysis" / "figs"
figs_path.mkdir(parents=True, exist_ok=True)

COLORS = {
    "base": PLOTCOLORS["dirt road"],
    "ours": PLOTCOLORS["concrete"],
}

In [ ]:
# Define channels
columns = {
    "imu": {
        "wx": True,
        "wy": True,
        "wz": True,
        "ax": True,
        "ay": True,
        "az": True,
    },
    "pro": {
        "velL": True,
        "velR": True,
        "curL": True,
        "curR": True,
    },
}
summary = pd.DataFrame({"columns": pd.Series(columns)})

In [ ]:
husky_summ = summary.copy()
vulpi_summ = summary.copy()

husky_dfs = preprocessing.get_recordings(husky_data, husky_summ)
vulpi_dfs = preprocessing.get_recordings(vulpi_data, vulpi_summ)

husky_pro = husky_dfs["pro"].copy()
vulpi_pro = vulpi_dfs["pro"].copy()

In [ ]:
husky_dfs["pro"]

In [ ]:
husky_pro["vx"], husky_pro["wz"] = transforms.unicycle_model(husky_pro)
vulpi_pro["vx"], vulpi_pro["wz"] = transforms.unicycle_model(vulpi_pro)

In [ ]:
with plt.style.context("analysis/figure.mplstyle"):
    fig, ax = plt.subplots()
    ax.scatter(
        vulpi_pro.wz,
        vulpi_pro.vx,
        c=COLORS["base"],
        s=0.5,
        alpha=1,
        label="Baseline",
        zorder=5,
        rasterized=True,
    )
    ax.scatter(
        husky_pro.wz,
        husky_pro.vx,
        c=COLORS["ours"],
        s=0.5,
        alpha=0.8,
        label="Ours",
        zorder=-1,
        rasterized=True,
    )

    ax.legend(loc="center left", bbox_to_anchor=(1, 0.5), markerscale=10)

    ax.set_xlabel("Yaw rate $\omega_z$ (rad/s)")
    ax.set_ylabel("Linear speed $v_x$ (m/s)")

    ax.set_aspect("equal")
    ax.set_xlim((-0.8, 0.8))
    ax.set_ylim((-0.8, 0.8))
    # ax.set_xlim((-0.05, 0.8))
    # ax.set_ylim((-0.05, 0.8))

    fig.savefig(figs_path / "speeds.pdf", bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots()
for terr in husky_pro.terrain.unique():
    # if terr in ["ICE"]:
    #     continue
    lab = terr.replace("_", " ")
    if terr == "ASPHALT":
        ax.scatter(
            husky_pro[husky_pro.terrain == terr].wz,
            husky_pro[husky_pro.terrain == terr].vx,
            c=PLOTCOLORS[lab.lower()],
            s=0.5,
            alpha=0.8,
            label=lab.title(),
            zorder=-1,
            rasterized=True,
        )

ax.legend(loc="center left", bbox_to_anchor=(1, 0.5), markerscale=10)

ax.set_xlabel("Yaw rate $\omega_z$ (rad/s)")
ax.set_ylabel("Linear speed $v_x$ (m/s)")

ax.set_aspect("equal")
ax.set_xlim((-0.8, 0.8))
ax.set_ylim((-0.8, 0.8))
# ax.set_xlim((-0.05, 0.8))
# ax.set_ylim((-0.05, 0.8))
plt.show()

In [ ]:
fig, ax = plt.subplots()
for terr in vulpi_pro.terrain.unique():
    # if terr in ["ICE"]:
    #     continue
    lab = terr.replace("_", " ")
    if lab == "CONCRETE":
        ax.scatter(
            vulpi_pro[vulpi_pro.terrain == terr].wz,
            vulpi_pro[vulpi_pro.terrain == terr].vx,
            c=PLOTCOLORS[lab.lower()],
            s=0.5,
            alpha=0.8,
            label=lab.title(),
            zorder=-1,
            rasterized=True,
        )

ax.legend(loc="center left", bbox_to_anchor=(1, 0.5), markerscale=10)

ax.set_xlabel("Yaw rate $\omega_z$ (rad/s)")
ax.set_ylabel("Linear speed $v_x$ (m/s)")

ax.set_aspect("equal")
ax.set_xlim((-0.8, 0.8))
ax.set_ylim((-0.8, 0.8))
# ax.set_xlim((-0.05, 0.8))
# ax.set_ylim((-0.05, 0.8))
plt.show()

In [ ]:
descriptions = {
    "hvx": stats.describe(husky_pro.vx)._asdict(),
    "hwz": stats.describe(husky_pro.wz)._asdict(),
    "vvx": stats.describe(vulpi_pro.vx)._asdict(),
    "vwz": stats.describe(vulpi_pro.wz)._asdict(),
}
pd.DataFrame.from_dict(descriptions, orient="index")

# Rotationnality

In [ ]:
from utils.constants import HuskyConstants

Bs = HuskyConstants.ugv_Bs
husky_pro["rho"] = (
    Bs * husky_pro.wz.abs() / (husky_pro.vx.abs() + Bs * husky_pro.wz.abs())
)
vulpi_pro["rho"] = (
    Bs * vulpi_pro.wz.abs() / (vulpi_pro.vx.abs() + Bs * vulpi_pro.wz.abs())
)

In [ ]:
fig, ax = plt.subplots()
ax.hist(
    [vulpi_pro.rho, husky_pro.rho],
    label=["Baseline", "Ours"],
    color=[COLORS["base"], COLORS["ours"]],
    log=True,
)
ax.legend()
ax.set_xlabel("Rotationnality $\\rho$")
ax.set_ylabel("Counts")

In [ ]:
rho_desc = {
    "ours": stats.describe(husky_pro.rho.dropna())._asdict(),
    "baseline": stats.describe(vulpi_pro.rho.dropna())._asdict(),
}
pd.DataFrame.from_dict(rho_desc, orient="index")